## Step 0: Setup Spark

In [1]:
import os

master = '--master local[1]'
#master = '--master spark://apachespark-master-2-1-0:7077`

conf = '--conf spark.cores.max=1 --conf spark.executor.memory=512m'

packages = '--packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1'

jars = '--jars /root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar'

py_files = '--py-files /root/lib/jpmml.py'

os.environ['PYSPARK_SUBMIT_ARGS'] = master \
  + ' ' + conf \
  + ' ' + packages \
  + ' ' + jars \
  + ' ' + py_files \
  + ' ' + 'pyspark-shell'

print(os.environ['PYSPARK_SUBMIT_ARGS'])

--master local[1] --conf spark.cores.max=1 --conf spark.executor.memory=512m --packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1 --jars /root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar --py-files /root/lib/jpmml.py pyspark-shell


### Import Spark Libraries

In [1]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import LinearRegression

### Configure Spark for Your Notebook
* You may need to Reconnect and/or Restart the Kernel to pick up changes.
* This examples uses the local Spark Master `--master local[1]`
* In production, you would use the PipelineIO Spark Master `--master spark://apachespark-master-2-1-0:7077`

### Create Spark Session
This may take a minute or two.  Please be patient.

In [3]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder.getOrCreate()

## Step 1: Load Training Data into Spark Cluster

### Read Data from Public S3 Bucket
* AWS credentials are not needed.
* We're asking Spark to infer the schema
* The data has a header
* Using `bzip2` because it's a splittable compression file format

In [6]:
df = spark_session.read.format("csv") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("s3a://datapalooza/airbnb/airbnb.csv.bz2")


print(df.head())

Row(id=5731498, name='A 2-bdrm house in Plaka of Athens', space='Ideally located a unique house in a very peaceful neighborhood of Plaka, near Acropolis. It is a traditional house in the heart of the historical center of Athens, in Plaka. The kitchen is fully equipped with oven, fridge with freezer. Cutlery, dishes and pans, kettle, espresso coffee maker (espresso capsules are provided), toaster. There is also a vacuum cleaner and a laundry machine. One big closet will make your stay more comfortable. Bed linen, towels and bath amenities are provided. Moreover, the apartment is fully airconditioned. The apartment is very close to a greek traditional tavernas, a pharmacy, banks and public transport.  Airport or any other transport is available upon demand at an additional but very reasonable cost. ', price='120.0', bathrooms='1.0', bedrooms='2.0', room_type='Entire home/apt', square_feet=None, host_is_super_host='0.0', city='Athina', state=None, cancellation_policy='moderate', security_

In [7]:
print(df.count())

198576


### Clean, Filter, and Summarize the Data

In [10]:
df_filtered = df.filter("price >= 50 AND price <= 750 AND bathrooms > 0.0 AND bedrooms is not null")

df_filtered.registerTempTable("df_filtered")

df_final = spark_session.sql("""
    select
        id,
        city,
        case when state in('NY', 'CA', 'London', 'Berlin', 'TX' ,'IL', 'OR', 'DC', 'WA')
            then state
            else 'Other'
        end as state,
        space,
        cast(price as double) as price,
        cast(bathrooms as double) as bathrooms,
        cast(bedrooms as double) as bedrooms,
        room_type,
        host_is_super_host,
        cancellation_policy,
        cast(case when security_deposit is null
            then 0.0
            else security_deposit
        end as double) as security_deposit,
        price_per_bedroom,
        cast(case when number_of_reviews is null
            then 0.0
            else number_of_reviews
        end as double) as number_of_reviews,
        cast(case when extra_people is null
            then 0.0
            else extra_people
        end as double) as extra_people,
        instant_bookable,
        cast(case when cleaning_fee is null
            then 0.0
            else cleaning_fee
        end as double) as cleaning_fee,
        cast(case when review_scores_rating is null
            then 80.0
            else review_scores_rating
        end as double) as review_scores_rating,
        cast(case when square_feet is not null and square_feet > 100
            then square_feet
            when (square_feet is null or square_feet <=100) and (bedrooms is null or bedrooms = 0)
            then 350.0
            else 380 * bedrooms
        end as double) as square_feet
    from df_filtered
""").persist()

df_final.registerTempTable("df_final")

df_final.select("square_feet", "price", "bedrooms", "bathrooms", "cleaning_fee").describe().show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44430)
Traceback (most recent call last):
  File "/root/spark-2.1.0-bin-fluxcapacitor/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/spark-2.1.0-bin-fluxcapacitor/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44430)

In [9]:
print(df_final.count())

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44430)
Traceback (most recent call last):
  File "/root/spark-2.1.0-bin-fluxcapacitor/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/spark-2.1.0-bin-fluxcapacitor/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44430)

In [ ]:
print(df_final.schema)

In [11]:
# Most popular cities

spark_session.sql("""
    select 
        state,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by state
    order by count(*) desc
""").show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44430)
Traceback (most recent call last):
  File "/root/spark-2.1.0-bin-fluxcapacitor/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/spark-2.1.0-bin-fluxcapacitor/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44430)

In [ ]:
# Most expensive popular cities

spark_session.sql("""
    select 
        city,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by city
    order by avg(price) desc
""").filter("ct > 25").show()

## Step 2: Feature Engineering

In [ ]:
continuous_features = ["bathrooms", \
                       "bedrooms", \
                       "security_deposit", \
                       "cleaning_fee", \
                       "extra_people", \
                       "number_of_reviews", \
                       "square_feet", \
                       "review_scores_rating"]

categorical_features = ["room_type", \
                        "host_is_super_host", \
                        "cancellation_policy", \
                        "instant_bookable", \
                        "state"]

### Continuous Features

In [ ]:
continuous_feature_assembler = VectorAssembler(inputCols=continuous_features, outputCol="unscaled_continuous_features")

continuous_feature_scaler = StandardScaler(inputCol="unscaled_continuous_features", outputCol="scaled_continuous_features", \
                                           withStd=True, withMean=False)

### Categorical Features

In [ ]:
categorical_feature_indexers = [StringIndexer(inputCol=x, \
                                              outputCol="{}_index".format(x)) \
                                for x in categorical_features]

categorical_feature_one_hot_encoders = [OneHotEncoder(inputCol=x.getOutputCol(), \
                                                      outputCol="oh_encoder_{}".format(x.getOutputCol() )) \
                                        for x in categorical_feature_indexers]

### Assemble Continuous and Categorical Features

In [ ]:
feature_cols_lr = [x.getOutputCol() \
                   for x in categorical_feature_one_hot_encoders]
feature_cols_lr.append("scaled_continuous_features")

feature_assembler_lr = VectorAssembler(inputCols=feature_cols_lr, \
                                       outputCol="features_lr")

## Step 3: Train the Pipeline

In [ ]:
linear_regression = LinearRegression(featuresCol="features_lr", \
                                     labelCol="price", \
                                     predictionCol="price_prediction", \
                                     maxIter=10, \
                                     regParam=0.3, \
                                     elasticNetParam=0.8)

estimators_lr = \
  [continuous_feature_assembler, continuous_feature_scaler] \
  + categorical_feature_indexers + categorical_feature_one_hot_encoders \
  + [feature_assembler_lr] + [linear_regression]

pipeline = Pipeline(stages=estimators_lr)

pipeline_model = pipeline.fit(df_final)

print(pipeline_model)

In [ ]:
pipeline_model.write().overwrite().save("model.parquet")

In [ ]:
type(df_final.schema)

In [ ]:
import dill as pickle

In [ ]:
with open('model.schema', 'wb') as pkl_file:
    pickle.dump(df_final.schema, pkl_file)

In [ ]:
!cat airbnb.schema

## SERVE FROM HERE

In [ ]:
restored_pipeline_model = PipelineModel.read().load("model.parquet")

In [ ]:
print(restored_pipeline_model.schema)

## Step 4:  Export the Pipeline Model

In [ ]:
df_final.schema.json()

In [ ]:
from pyspark.sql.types import _parse_datatype_json_string
schema = _parse_datatype_json_string(df_final.schema.json())
print(schema)

In [ ]:
from pyspark.ml.common import _py2java

schema_as_java = _py2java(spark_session, schema)
print(schema_as_java)

In [ ]:
from pyspark.ml.common import _py2java
from pyspark.ml import PipelineModel
from pyspark.sql.types import StructType
import dill as pickle

spark_df_schema_as_json = df_final.schema.toInternal(df_final.schema)
with open('model.schema', 'wb') as pkl_file:
    pickle.dump(spark_df_schema_as_json, pkl_file)

pipeline_model.write().overwrite().save('model.parquet')

## SERVE FROM HERE
with open('model.schema', 'rb') as pkl_file:
    from pyspark.sql.types import _parse_datatype_json_string
    restored_spark_df_schema_as_json = pickle.load(pkl_file)
    print(type(restored_spark_df_schema_as_json))
    restored_spark_df_schema = restored_spark_df_schema_as_json.fromInternal(restored_spark_df_schema_as_json)
#_parse_datatype_json_string(restored_spark_df_schema_as_json)
    #print(restored_spark_df_schema)
#   restored_spark_df_schema_as_java = _py2java(spark_session, restored_spark_df_schema)

restored_spark_pipeline_model = PipelineModel.read().load('model.parquet')
restored_spark_pipeline_model_as_java = restored_spark_pipeline_model._to_java()

print(spark_session._jvm.org.jpmml.sparkml.ConverterUtil.toPMMLByteArray(restored_spark_df_schema,
                                                                          restored_spark_pipeline_model_as_java))

In [14]:
import pio_bundler

pio_bundler.bundle(spark_session, df_final, pipeline_model_restore)

NameError: name 'pipeline_model_restore' is not defined

In [ ]:
from jpmml import toPMMLBytes

model = toPMMLBytes(spark_session, df_final, pipeline_model_restore)

with open('airbnb.model', 'wb') as fh:
    fh.write(model)

In [ ]:
!cat airbnb.model

## Step 5:  Deploy the Pipeline Model

### Option 1 of 2:  Deploy with PipelineIO CLI!

In [ ]:
%%bash

pip install -q pio-cli==0.37

Configure CLI for Model Deployment

In [ ]:
%%bash

pio init-model --model-server-url=http://prediction-pmml.demo.pipeline.io --model-type=pmml \
    --model-namespace=cli --model-name=airbnb

In [ ]:
%%bash

pio deploy --model-version=v0 airbnb.model

### Deployment Option 2:  REST API

In [ ]:
import requests

deploy_url = 'http://prediction-pmml.demo.pipeline.io/api/v1/model/deploy/pmml/rest/airbnb/v0'

files = {'file': open('airbnb.model', 'rb')}

response = requests.post(deploy_url, files=files)

print("Success! %s" % response.text)

## Step 6:  Predict With Deployed Pipeline Model

### Setup Prediction Inputs

In [ ]:
import json

data = {"bathrooms":2.0, 
        "bedrooms":2.0, 
        "security_deposit":175.00, 
        "cleaning_fee":25.0, 
        "extra_people":1.0, 
        "number_of_reviews": 2.0, 
        "square_feet": 250.0, 
        "review_scores_rating": 2.0, 
        "room_type": "Entire home/apt", 
        "host_is_super_host": "0.0", 
        "cancellation_policy": "flexible", 
        "instant_bookable": "1.0", 
        "state": "CA" }

json_data = json.dumps(data)

with open('airbnb-predict-inputs.json', 'wt') as fh:
    fh.write(json_data)

### Predict with CLI
Note:  Run again if you see a fallback on the first try. (bug)

In [ ]:
%%bash

pio predict --model-version=v0 \
            --model-input-filename=airbnb-predict-inputs.json

### Predict with REST

In [ ]:
import json

# Note:  You may need to run this twice.
#        A fallback may trigger the first time due to lazy loading of models.
predict_url = 'http://prediction-pmml.demo.pipeline.io/api/v1/model/predict/pmml/rest/airbnb/v0'

headers = {'content-type': 'application/json'}

response = requests.post(predict_url, 
                         data=json_data, 
                         headers=headers)

print(response.text)

## Step 7:  Monitor Model Servers through Dashboards

### Fallbacks and Circuit Breaker [Dashboard](http://hystrix.community.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Model%20Servers%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine.community.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D)

In [ ]:
%%html

<iframe width=800 height=600 src="http://hystrix.community.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Model%20Servers%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine.community.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D"></iframe>